# Turtlebot Teleop Demo

Make sure there is a robot published, you can launch turtlebot3 waffle with the following commands:

`export TURTLEBOT3_MODEL=waffle`

`roslaunch turtlebot3_fake turtlebot3_fake.launch`

In [ ]:
import ipywidgets as wd
from ipywidgets import FloatSlider, Layout, Button, Box, FloatText, Label

In [ ]:
from geometry_msgs.msg import Twist
import rospy

rospy.init_node('commander')

In [ ]:
import jupyros

In [ ]:
linear_slider = FloatSlider(min = -1, max = 1, step = 0.05)
angular_slider = FloatSlider(min = -1, max = 1, step = 0.1)

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

In [ ]:
%%thread_cell

rate = rospy.Rate(5)
while True:
    msg = Twist()
    msg.linear.x = linear_slider.value
    msg.angular.z = angular_slider.value
    pub.publish(msg)
    rate.sleep()

In [ ]:
controls = [ Box([Label(value="Linear"), linear_slider]), Box([Label(value="Angular"), angular_slider])]

form = Box(controls, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    padding='10px'
))

form